# AgenticNewsvendor - Data Exploration and Testing

This notebook demonstrates the core functionality of the AgenticNewsvendor research project, including:
- Data generation for newsvendor problems
- Agentic system integration with LangChain
- Dashboard visualization capabilities
- End-to-end workflow testing

## 1. Setup and Imports

In [ ]:
# Import standard libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Add project root to path
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("✅ Basic libraries imported successfully")
print(f"Project root: {project_root}")

: 

In [ ]:
# Import project modules
try:
    from data_generation import DataGenerator
    from agentic_system import NewsvendorAgent, AgentManager, create_agent_manager_with_default_agents
    print("✅ Project modules imported successfully")
except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Make sure you're running this notebook from the notebooks/ directory")

## 2. Data Generation

Let's generate synthetic demand data for our newsvendor analysis.

In [ ]:
# Initialize data generator
data_gen = DataGenerator(seed=42)

# Generate demand data for 5 products over 50 periods
demand_data = data_gen.generate_demand_data(n_products=5, n_periods=50)

print(f"Generated demand data shape: {demand_data.shape}")
print("\nFirst few rows:")
demand_data.head(10)

In [ ]:
# Generate time series data
time_series_data = data_gen.generate_time_series(
    start_date='2024-01-01',
    end_date='2024-12-31',
    frequency='D',
    noise_level=0.15
)

print(f"Generated time series data shape: {time_series_data.shape}")
print("\nFirst few rows:")
time_series_data.head()

## 3. Data Exploration and Visualization

In [ ]:
# Basic statistics for demand data
print("=== Demand Data Statistics ===")
print(demand_data.groupby('product')['demand'].describe())

In [ ]:
# Visualize demand patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Demand distribution
axes[0, 0].hist(demand_data['demand'], bins=30, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Overall Demand Distribution')
axes[0, 0].set_xlabel('Demand')
axes[0, 0].set_ylabel('Frequency')

# Demand by product
sns.boxplot(data=demand_data, x='product', y='demand', ax=axes[0, 1])
axes[0, 1].set_title('Demand Distribution by Product')
axes[0, 1].tick_params(axis='x', rotation=45)

# Demand over time for first product
product_1_data = demand_data[demand_data['product'] == 'Product_001']
axes[1, 0].plot(product_1_data['period'], product_1_data['demand'], marker='o', markersize=3)
axes[1, 0].set_title('Demand Over Time (Product_001)')
axes[1, 0].set_xlabel('Period')
axes[1, 0].set_ylabel('Demand')

# Time series visualization
axes[1, 1].plot(time_series_data['timestamp'][:100], time_series_data['value'][:100])
axes[1, 1].set_title('Time Series Data (First 100 Days)')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Value')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Agentic System Testing

Now let's test our agentic system with newsvendor optimization tasks.

In [ ]:
# Create agent manager with default agents
agent_manager = create_agent_manager_with_default_agents()

print("Available agents:")
for agent_name in agent_manager.get_available_agents():
    print(f"  - {agent_name}")

In [ ]:
# Test demand analysis
product_demand = demand_data[demand_data['product'] == 'Product_001']['demand'].tolist()

analysis_context = {
    'demand_data': product_demand
}

analysis_result = agent_manager.execute_task(
    agent_name='NewsvendorAgent',
    task='analyze',
    context=analysis_context
)

print("=== Demand Analysis Results ===")
for key, value in analysis_result.items():
    print(f"{key}: {value}")

In [ ]:
# Test inventory optimization
optimization_context = {
    'demand_data': product_demand,
    'unit_cost': 8.0,
    'selling_price': 12.0,
    'salvage_value': 3.0
}

optimization_result = agent_manager.execute_task(
    agent_name='NewsvendorAgent',
    task='optimize',
    context=optimization_context
)

print("=== Inventory Optimization Results ===")
for key, value in optimization_result.items():
    print(f"{key}: {value}")

In [ ]:
# Test demand forecasting
forecast_context = {
    'demand_data': product_demand,
    'forecast_periods': 5
}

forecast_result = agent_manager.execute_task(
    agent_name='NewsvendorAgent',
    task='forecast',
    context=forecast_context
)

print("=== Demand Forecasting Results ===")
for key, value in forecast_result.items():
    print(f"{key}: {value}")

## 5. Multi-Product Optimization Analysis

In [ ]:
# Analyze all products
products = demand_data['product'].unique()
optimization_results = []

for product in products:
    product_demand = demand_data[demand_data['product'] == product]['demand'].tolist()
    
    context = {
        'demand_data': product_demand,
        'unit_cost': np.random.uniform(5, 10),  # Random costs for variety
        'selling_price': np.random.uniform(12, 20),
        'salvage_value': np.random.uniform(1, 5)
    }
    
    result = agent_manager.execute_task('NewsvendorAgent', 'optimize', context)
    result['product'] = product
    optimization_results.append(result)

# Create results DataFrame
results_df = pd.DataFrame(optimization_results)
print("=== Multi-Product Optimization Results ===")
results_df[['product', 'optimal_quantity', 'critical_ratio', 'unit_cost', 'selling_price']].round(2)

In [ ]:
# Visualize optimization results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Optimal quantities
axes[0].bar(results_df['product'], results_df['optimal_quantity'])
axes[0].set_title('Optimal Order Quantities by Product')
axes[0].set_xlabel('Product')
axes[0].set_ylabel('Optimal Quantity')
axes[0].tick_params(axis='x', rotation=45)

# Critical ratios
axes[1].bar(results_df['product'], results_df['critical_ratio'])
axes[1].set_title('Critical Ratios by Product')
axes[1].set_xlabel('Product')
axes[1].set_ylabel('Critical Ratio')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Dashboard Integration Preview

Let's test the dashboard components that will be used in the Streamlit app.

In [ ]:
# Test dashboard functionality (without Streamlit dependencies)
try:
    from dashboard import Dashboard
    
    dashboard = Dashboard("Test Dashboard")
    
    # Calculate metrics
    metrics = dashboard.create_metrics_dashboard(demand_data)
    
    print("=== Dashboard Metrics ===")
    for key, value in metrics.items():
        print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")
        
except ImportError as e:
    print(f"Dashboard components not available (expected in notebook environment): {e}")
    
    # Create manual metrics calculation
    metrics = {
        'total_demand': demand_data['demand'].sum(),
        'avg_demand': demand_data['demand'].mean(),
        'demand_volatility': demand_data['demand'].std(),
        'num_products': demand_data['product'].nunique(),
        'num_periods': demand_data['period'].nunique()
    }
    
    print("=== Manual Dashboard Metrics ===")
    for key, value in metrics.items():
        print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")

## 7. Data Export for Dashboard

In [ ]:
# Export data for dashboard testing
output_dir = '../data'
os.makedirs(output_dir, exist_ok=True)

# Export demand data
demand_file = os.path.join(output_dir, 'sample_demand_data.csv')
data_gen.export_data(demand_data, demand_file, 'csv')

# Export time series data
ts_file = os.path.join(output_dir, 'sample_timeseries_data.csv')
data_gen.export_data(time_series_data, ts_file, 'csv')

# Export optimization results
results_file = os.path.join(output_dir, 'optimization_results.csv')
results_df.to_csv(results_file, index=False)

print(f"✅ Data exported to {output_dir}/")
print(f"  - {demand_file}")
print(f"  - {ts_file}")
print(f"  - {results_file}")

## 8. Summary and Next Steps

### What we've accomplished:
1. ✅ Successfully generated synthetic demand data
2. ✅ Tested agentic system with newsvendor optimization
3. ✅ Performed multi-product analysis
4. ✅ Exported data for dashboard integration

### Next steps:
1. **Install required packages**: Run `pip install -r requirements.txt`
2. **Test Streamlit dashboard**: Run `streamlit run dashboard/__init__.py`
3. **Enhance with LangChain**: Add proper LangChain integration after installing langchain packages
4. **Docker setup**: Create Docker configuration for deployment
5. **Advanced optimization**: Implement more sophisticated optimization algorithms

In [ ]:
# Final verification
print("=== Final System Check ===")
print(f"✅ Data generation: {len(demand_data)} demand records created")
print(f"✅ Agentic system: {len(agent_manager.get_task_history())} tasks executed")
print(f"✅ Multi-product optimization: {len(results_df)} products analyzed")
print(f"✅ Data export: 3 files created in {output_dir}/")
print("\n🎉 AgenticNewsvendor system is ready for further development!")